In [1]:

import numpy as np  
import pandas as pd 
import os 
import re
import warnings
from tqdm import tqdm
import datetime as dt

# ML/DL imports
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import keras

# Plotting imports
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

from numpy.random import seed
seed(1)

# Allows us to see more information regarding the DataFrame
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


In [2]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

# Reading in the datasets

In [3]:
path = '../datasets/m5-forecasting-accuracy/'

In [4]:

train_sales = pd.read_csv(path+'sales_train_validation.csv')
calendar = pd.read_csv(path+'calendar.csv')
sell_prices = pd.read_csv(path+'sell_prices.csv')
#df_val = pd.read_csv(path+'sales_train_validation.csv')

In [5]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [5]:
downcast_dtypes(train_sales)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [6]:
downcast_dtypes(calendar)
downcast_dtypes(sell_prices)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [7]:
downcast_dtypes(calendar)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [7]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int16(1913), object(6)
memory usage: 112.6+ MB


# Preparing the Data

In [4]:
train_sales.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,d_245,d_246,d_247,d_248,d_249,d_250,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30

In [5]:
# d_1, d_2, …, d_i, … d_1941: The number of units sold at day i, starting from 2011-01-29.


train_sales

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [6]:
#date: The date in a “y-m-d” format.

# wm_yr_wk: The id of the week the date belongs to.

# weekday: The type of the day (Saturday, Sunday, …, Friday).

# wday: The id of the weekday, starting from Saturday.

# month: The month of the date.

# year: The year of the date.
# snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP3 purchases on the examined date. 1 indicates that SNAP purchases are allowed.


calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [7]:
calendar.describe()

,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI
count,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000,1969.000000
mean,11347.086338,3.997461,6.325546,2013.288471,0.330117,0.330117,0.330117
std,155.277043,2.001141,3.416864,1.580198,0.470374,0.470374,0.470374
min,11101.000000,1.000000,1.000000,2011.000000,0.000000,0.000000,0.000000
25%,11219.000000,2.000000,3.000000,2012.000000,0.000000,0.000000,0.000000
50%,11337.000000,4.000000,6.000000,2013.000000,0.000000,0.000000,0.000000
75%,11502.000000,6.000000,9.000000,2015.000000,1.000000,1.000000,1.000000
max,11621.000000,7.000000,12.000000,2016.000000,1.000000,1.000000,1.000000


In [8]:
#sell_price: The price of the product for the given week/store. 

#The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. 
#Note that although prices are constant at weekly basis, they may change through time (both training and test set).



sell_prices

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [9]:
sell_prices.describe()

,wm_yr_wk,sell_price
count,6.841121e+06,6.841121e+06
mean,1.138294e+04,4.410952e+00
std,1.486100e+02,3.408814e+00
min,1.110100e+04,1.000000e-02
25%,1.124700e+04,2.180000e+00
50%,1.141100e+04,3.470000e+00
75%,1.151700e+04,5.840000e+00
max,1.162100e+04,1.073200e+02


In [10]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [11]:
calendar.isna().sum()

date               0
wm_yr_wk           0
weekday            0
wday               0
month              0
year               0
d                  0
event_name_1    1807
event_type_1    1807
event_name_2    1964
event_type_2    1964
snap_CA            0
snap_TX            0
snap_WI            0
dtype: int64

In [12]:
train_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1919 entries, id to d_1913
dtypes: int64(1913), object(6)
memory usage: 446.4+ MB


In [13]:
train_sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,d_46,d_47,d_48,d_49,d_50,d_51,d_52,d_53,d_54,d_55,d_56,d_57,d_58,d_59,d_60,d_61,d_62,d_63,d_64,d_65,d_66,d_67,d_68,d_69,d_70,d_71,d_72,d_73,d_74,d_75,d_76,d_77,d_78,d_79,d_80,d_81,d_82,d_83,d_84,d_85,d_86,d_87,d_88,d_89,d_90,d_91,d_92,d_93,d_94,d_95,d_96,d_97,d_98,d_99,d_100,d_101,d_102,d_103,d_104,d_105,d_106,d_107,d_108,d_109,d_110,d_111,d_112,d_113,d_114,d_115,d_116,d_117,d_118,d_119,d_120,d_121,d_122,d_123,d_124,d_125,d_126,d_127,d_128,d_129,d_130,d_131,d_132,d_133,d_134,d_135,d_136,d_137,d_138,d_139,d_140,d_141,d_142,d_143,d_144,d_145,d_146,d_147,d_148,d_149,d_150,d_151,d_152,d_153,d_154,d_155,d_156,d_157,d_158,d_159,d_160,d_161,d_162,d_163,d_164,d_165,d_166,d_167,d_168,d_169,d_170,d_171,d_172,d_173,d_174,d_175,d_176,d_177,d_178,d_179,d_180,d_181,d_182,d_183,d_184,d_185,d_186,d_187,d_188,d_189,d_190,d_191,d_192,d_193,d_194,d_195,d_196,d_197,d_198,d_199,d_200,d_201,d_202,d_203,d_204,d_205,d_206,d_207,d_208,d_209,d_210,d_211,d_212,d_213,d_214,d_215,d_216,d_217,d_218,d_219,d_220,d_221,d_222,d_223,d_224,d_225,d_226,d_227,d_228,d_229,d_230,d_231,d_232,d_233,d_234,d_235,d_236,d_237,d_238,d_239,d_240,d_241,d_242,d_243,d_244,...,d_1664,d_1665,d_1666,d_1667,d_1668,d_1669,d_1670,d_1671,d_1672,d_1673,d_1674,d_1675,d_1676,d_1677,d_1678,d_1679,d_1680,d_1681,d_1682,d_1683,d_1684,d_1685,d_1686,d_1687,d_1688,d_1689,d_1690,d_1691,d_1692,d_1693,d_1694,d_1695,d_1696,d_1697,d_1698,d_1699,d_1700,d_1701,d_1702,d_1703,d_1704,d_1705,d_1706,d_1707,d_1708,d_1709,d_1710,d_1711,d_1712,d_1713,d_1714,d_1715,d_1716,d_1717,d_1718,d_1719,d_1720,d_1721,d_1722,d_1723,d_1724,d_1725,d_1726,d_1727,d_1728,d_1729,d_1730,d_1731,d_1732,d_1733,d_1734,d_1735,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,d_1746,d_1747,d_1748,d_1749,d_1750,d_1751,d_1752,d_1753,d_1754,d_1755,d_1756,d_1757,d_1758,d_1759,d_1760,d_1761,d_1762,d_1763,d_1764,d_1765,d_1766,d_1767,d_1768,d_1769,d_1770,d_1771,d_1772,d_1773,d_1774,d_1775,d_1776,d_1777,d_1778,d_1779,d_1780,d_1781,d_1782,d_1783,d_1784,d_1785,d_1786,d_1787,d_1788,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798,d_1799,d_1800,d_1801,d_1802,d_1803,d_1804,d_1805,d_1806,d_1807,d_1808,d_1809,d_1810,d_1811,d_1812,d_1813,d_1814,d_1815,d_1816,d_1817,d_1818,d_1819,d_1820,d_1821,d_1822,d_1823,d_1824,d_1825,d_1826,d_1827,d_1828,d_1829,d_1830,d_1831,d_1832,d_1833,d_1834,d_1835,d_1836,d_1837,d_1838,d_1839,d_1840,d_1841,d_1842,d_1843,d_1844,d_1845,d_1846,d_1847,d_1848,d_1849,d_1850,d_1851,d_1852,d_1853,d_1854,d_1855,d_1856,d_1857,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,2,0,2,2,0,0,0,1,1,0,2,0,1,1,2,0,1,0,0,0,2,1,0,1,1,2,0,0,0,0,0,0,0,1,0,1,0,1,0,3,1,1,0,1,1,2,0,0,0,0,1,1,0,0,0,0,3,0,1,0,0,0,0,1,1,1,0,1,0,2,0,0,0,0,2,0,0,0,0,1,1,2,0,0,0,0,2,0,0,1,1,1,1,0,0,0,0,0,1,2,2,0,1,0,0,0,0,1,2,1,0,0,0,0,0,1,0,3,0,1,2,1,0,3,0,0,0,1,0,2,2,1,0,0,1,2,0,1

In [8]:
# merging calendar and price data on week, month and year
# The price is provided per week (average across seven days)

sell_prices_calendar = pd.merge(sell_prices, calendar, on = 'wm_yr_wk')
#sell_prices_calendar.head()

In [15]:
len(sell_prices_calendar)

47735397

In [15]:
sell_prices_calendar.head()

,store_id,item_id,wm_yr_wk,sell_price,date,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,CA_1,HOBBIES_1_001,11325,9.58,2013-07-13,Saturday,1,7,2013,d_897,NaN,NaN,NaN,NaN,0,1,0
1,CA_1,HOBBIES_1_001,11325,9.58,2013-07-14,Sunday,2,7,2013,d_898,NaN,NaN,NaN,NaN,0,0,1
2,CA_1,HOBBIES_1_001,11325,9.58,2013-07-15,Monday,3,7,2013,d_899,NaN,NaN,NaN,NaN,0,1,1
3,CA_1,HOBBIES_1_001,11325,9.58,2013-07-16,Tuesday,4,7,2013,d_900,NaN,NaN,NaN,NaN,0,0,0
4,CA_1,HOBBIES_1_001,11325,9.58,2013-07-17,Wednesday,5,7,2013,d_901,NaN,NaN,NaN,NaN,0,0,0


In [9]:
reduced_sell_prices_calendar = sell_prices_calendar.drop(['wm_yr_wk','weekday','wday','month','year','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2'], axis=1)

In [ ]:
len(reduced_sell_prices_calendar)

In [ ]:
#reduced_sell_prices_calendar_sort = reduced_sell_prices_calendar.sort_values(by = 'date')

In [ ]:
#reduced_sell_prices_calendar_sort

In [10]:
# changing the sales dataset to turn the 'd' columns to roll to help mearge the dataset,

train_sales_melt = train_sales.melt(id_vars=["id","item_id","dept_id", "cat_id","store_id","state_id"], var_name="d", value_name="sold_units")
train_sales_melt

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold_units
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
58327365,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1913,1
58327366,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327367,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1913,0
58327368,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1913,3


In [11]:
# merging the 3 data datasets together
sales_prices = pd.merge(reduced_sell_prices_calendar, train_sales_melt, on = ['item_id', 'store_id', 'd'])
sales_prices.head()

,store_id,item_id,sell_price,date,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units
0,CA_1,HOBBIES_1_001,9.58,2013-07-13,d_897,0,1,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0
1,CA_1,HOBBIES_1_001,9.58,2013-07-14,d_898,0,0,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0
2,CA_1,HOBBIES_1_001,9.58,2013-07-15,d_899,0,1,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0
3,CA_1,HOBBIES_1_001,9.58,2013-07-16,d_900,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0
4,CA_1,HOBBIES_1_001,9.58,2013-07-17,d_901,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0


In [12]:
# to convert the sell_price and sold_units to floats
sales_prices[['sell_price', 'sold_units']] = sales_prices[['sell_price', 'sold_units']].astype(float)

In [13]:
# creating a new column to calculate revenue
sales_prices['revenue'] = sales_prices['sell_price'] * sales_prices['sold_units']
sales_prices

,store_id,item_id,sell_price,date,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units,revenue
0,CA_1,HOBBIES_1_001,9.58,2013-07-13,d_897,0,1,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.0
1,CA_1,HOBBIES_1_001,9.58,2013-07-14,d_898,0,0,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.0
2,CA_1,HOBBIES_1_001,9.58,2013-07-15,d_899,0,1,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.0
3,CA_1,HOBBIES_1_001,9.58,2013-07-16,d_900,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.0
4,CA_1,HOBBIES_1_001,9.58,2013-07-17,d_901,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46027952,WI_3,FOODS_3_825,4.00,2011-02-28,d_31,0,0,0,FOODS_3_825_WI_3_validation,FOODS_3,FOODS,WI,1.0,4.0
46027953,WI_3,FOODS_3_825,4.00,2011-03-01,d_32,1,1,0,FOODS_3_825_WI_3_validation,FOODS_3,FOODS,WI,2.0,8.0
46027954,WI_3,FOODS_3_825,4.00,2011-03-02,d_33,1,0,1,FOODS_3_825_WI_3_validation,FOODS_3,FOODS,WI,2.0,8.0
46027955,WI_3,FOODS_3_825,4.00,2011-03-03,d_34,1,1,1,FOODS_3_825_WI_3_validation,FOODS_3,FOODS,WI,2.0,8.0


In [14]:
sales_prices.set_index('date', inplace=True)

In [15]:
sales_prices.head(10)

,store_id,item_id,sell_price,d,snap_CA,snap_TX,snap_WI,id,dept_id,cat_id,state_id,sold_units,revenue
date,,,,,,,,,,,,,
2013-07-13,CA_1,HOBBIES_1_001,9.58,d_897,0,1,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-14,CA_1,HOBBIES_1_001,9.58,d_898,0,0,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-15,CA_1,HOBBIES_1_001,9.58,d_899,0,1,1,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-16,CA_1,HOBBIES_1_001,9.58,d_900,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-17,CA_1,HOBBIES_1_001,9.58,d_901,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-18,CA_1,HOBBIES_1_001,9.58,d_902,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,1.0,9.58
2013-07-19,CA_1,HOBBIES_1_001,9.58,d_903,0,0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00
2013-07-13,CA_1,HOBBIES_1_002,3.97,d_897,0,1,0,HOBBIES_1_002_CA_1_validation,HOBBIES_1,HOBBIES,CA,1.0,3.97
2013-07-14,CA_1,HOBBIES_1_002,3.97,d_898,0,0,1,HOBBIES_1_002_CA_1_validation,HOBBIES_1,HOBBIES,CA,0.0,0.00


In [16]:
#sales_prices_drop = sales_prices.drop(['event_type_1', 'event_name_1','event_type_2', 'event_name_2'], axis = 1)

In [14]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
    data: Sequence of observations as a list or NumPy array.
    n_in: Number of lag observations as input (X).
    n_out: Number of observations as output (y).
    dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
    Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [23]:
values = [x for x in range(10)]
data = series_to_supervised(values)
print(data)

   var1(t-1)  var1(t)
1        0.0        1
2        1.0        2
3        2.0        3
4        3.0        4
5        4.0        5
6        5.0        6
7        6.0        7
8        7.0        8
9        8.0        9


In [24]:
data = series_to_supervised(values, 3)
data

,var1(t-3),var1(t-2),var1(t-1),var1(t)
3,0.0,1.0,2.0,3
4,1.0,2.0,3.0,4
5,2.0,3.0,4.0,5
6,3.0,4.0,5.0,6
7,4.0,5.0,6.0,7
8,5.0,6.0,7.0,8
9,6.0,7.0,8.0,9


In [25]:
data = series_to_supervised(values, 2, 2)
print(data)

   var1(t-2)  var1(t-1)  var1(t)  var1(t+1)
2        0.0        1.0        2        3.0
3        1.0        2.0        3        4.0
4        2.0        3.0        4        5.0
5        3.0        4.0        5        6.0
6        4.0        5.0        6        7.0
7        5.0        6.0        7        8.0
8        6.0        7.0        8        9.0


In [28]:
raw = DataFrame()
raw['ob1'] = [x for x in range(10)]
raw['ob2'] = [x for x in range(50, 60)]


In [29]:
raw

,ob1,ob2
0,0,50
1,1,51
2,2,52
3,3,53
4,4,54
5,5,55
6,6,56
7,7,57
8,8,58
9,9,59


In [30]:
values = raw.values


In [31]:
values

array([[ 0, 50],
       [ 1, 51],
       [ 2, 52],
       [ 3, 53],
       [ 4, 54],
       [ 5, 55],
       [ 6, 56],
       [ 7, 57],
       [ 8, 58],
       [ 9, 59]], dtype=int64)

In [32]:
data = series_to_supervised(values)
print(data)

   var1(t-1)  var2(t-1)  var1(t)  var2(t)
1        0.0       50.0        1       51
2        1.0       51.0        2       52
3        2.0       52.0        3       53
4        3.0       53.0        4       54
5        4.0       54.0        5       55
6        5.0       55.0        6       56
7        6.0       56.0        7       57
8        7.0       57.0        8       58
9        8.0       58.0        9       59


In [33]:
data = series_to_supervised(values, 1, 2)
print(data)

   var1(t-1)  var2(t-1)  var1(t)  var2(t)  var1(t+1)  var2(t+1)
1        0.0       50.0        1       51        2.0       52.0
2        1.0       51.0        2       52        3.0       53.0
3        2.0       52.0        3       53        4.0       54.0
4        3.0       53.0        4       54        5.0       55.0
5        4.0       54.0        5       55        6.0       56.0
6        5.0       55.0        6       56        7.0       57.0
7        6.0       56.0        7       57        8.0       58.0
8        7.0       57.0        8       58        9.0       59.0


In [ ]:
sales_prices_ana = sales_prices.sample(n = 3000000)

In [13]:
sales_prices_ana

NameError: name 'sales_prices_ana' is not defined

In [20]:
values = sales_prices_ana.values

In [21]:
len(sales_prices_ana.values)

3000000

In [22]:
values

array([['TX_3', 'FOODS_3_106', 1.0, ..., 'TX', 0.0, 0.0],
       ['CA_4', 'HOUSEHOLD_2_215', 1.93, ..., 'CA', 2.0, 3.86],
       ['CA_3', 'FOODS_2_333', 3.28, ..., 'CA', 3.0, 9.84],
       ...,
       ['WI_1', 'HOUSEHOLD_2_336', 9.26, ..., 'WI', 0.0, 0.0],
       ['CA_2', 'FOODS_3_563', 3.78, ..., 'CA', 0.0, 0.0],
       ['WI_2', 'HOUSEHOLD_2_255', 9.97, ..., 'WI', 0.0, 0.0]],
      dtype=object)

In [ ]:
sales_prices_ana.sort_values(by='date')

In [16]:
# transposing dataset
train_sales = train_sales.T

In [18]:
train_sales = train_sales[6:]

In [19]:
train_sales

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,30240,30241,30242,30243,30244,30245,30246,30247,30248,30249,30250,30251,30252,30253,30254,30255,30256,30257,30258,30259,30260,30261,30262,30263,30264,30265,30266,30267,30268,30269,30270,30271,30272,30273,30274,30275,30276,30277,30278,30279,30280,30281,30282,30283,30284,30285,30286,30287,30288,30289,30290,30291,30292,30293,30294,30295,30296,30297,30298,30299,30300,30301,30302,30303,30304,30305,30306,30307,30308,30309,30310,30311,30312,30313,30314,30315,30316,30317,30318,30319,30320,30321,30322,30323,30324,30325,30326,30327,30328,30329,30330,30331,30332,30333,30334,30335,30336,30337,30338,30339,30340,30341,30342,30343,30344,30345,30346,30347,30348,30349,30350,30351,30352,30353,30354,30355,30356,30357,30358,30359,30360,30361,30362,30363,30364,30365,30366,30367,30368,30369,30370,30371,30372,30373,30374,30375,30376,30377,30378,30379,30380,30381,30382,30383,30384,30385,30386,30387,30388,30389,30390,30391,30392,30393,30394,30395,30396,30397,30398,30399,30400,30401,30402,30403,30404,30405,30406,30407,30408,30409,30410,30411,30412,30413,30414,30415,30416,30417,30418,30419,30420,30421,30422,30423,30424,30425,30426,30427,30428,30429,30430,30431,30432,30433,30434,30435,30436,30437,30438,30439,30440,30441,30442,30443,30444,30445,30446,30447,30448,30449,30450,30451,30452,30453,30454,30455,30456,30457,30458,30459,30460,30461,30462,30463,30464,30465,30466,30467,30468,30469,30470,30471,30472,30473,30474,30475,30476,30477,30478,30479,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_1,0,0,0,0,0,0,0,12,2,0,0,0,0,0,4,5,0,0,0,0,0,2,2,0,0,0,0,0,2,0,0,9,0,0,0,2,0,0,0,0,0,0,0,3,0,0,1,0,0,2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,5,2,0,0,0,0,4,0,1,0,1,5,0,0,1,0,0,0,0,2,2,1,0,25,0,0,1,0,4,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4,0,0,0,0,0,0,0,0,0,0,2,0,0,3,0,0,0,0,0,3,2,0,0,0,0,0,0,0,2,0,0,0,0,3,0,0,0,0,45,0,0,1,0,0,0,0,0,10,0,0,0,0,0,4,0,0,0,8,0,0,0,3,5,0,0,0,0,0,3,2,0,0,0,0,0,0,4,0,0,0,3,0,0,7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,1,1,0,0,7,0,5,0,0,0,0,0,0,0,0,0,16,0,54,0,...,0,0,2,0,0,25,0,0,0,10,0,61,57,0,0,2,1,0,0,8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5,1,3,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,2,5,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,6,0,0,0,0,12,0,0,1,0,0,3,0,0,1,0,0,0,28,0,0,0,2,0,0,0,0,0,0,13,0,47,2,0,5,0,0,0,4,0,0,0,0,0,0,0,0,57,4,0,47,0,0,0,2,0,0,3,0,40,0,0,0,6,0,3,0,5,0,0,0,2,0,6,0,0,0,0,13,0,4,0,0,3,0,0,32,3,0,17,2,4,0,0,0,0,0,0,0,1,1,8,0,0,0,0,0,0,0,1,0,0,1,0,0,1,13,1,14,0,3,0,0,0,0,0,16,2,1,0,0,0,0,0,36,0,0,0,14,0,0,7,4,0,0,0,0,0,0,0,0,0,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,3,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,4,0,0,1,0,0,0,4,2,1,2,0,8,0,0,3,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0,2,0,0,0,0,0,0,0,0,1,0,2,0,12,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,15,0,0,0,0,0,0,0,0,19,0,0,0,0,0,0,0,1,0,3,0,0,0,0,5,0,0,0,0,0,3,0,0,0,0,0,1,0,4,0,0,0,0,0,0,5,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,8,0,0,0,0,0,0,1,0,0,0,0,8,0,15,0,...,0,0,1,0,0,31,1,0,0,11,1,67,62,0,0,0,0,0,0,1,0,4,0,0,0,0,0,

In [20]:
# scale the dataset becomes 0 to 1. By doing so, all features will be transformed into the range [0,1] meaning that the minimum and maximum value of a feature/variable is going to be 0 and 1, respectively so that they are in the same range.
sc = MinMaxScaler(feature_range=(0, 1))
train_sales = sc.fit_transform(train_sales)

In [21]:
print(train_sales)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.3        0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.16666667 ... 0.         0.25       0.        ]
 [0.2        0.         0.16666667 ... 0.05       0.08333333 0.        ]
 [0.2        0.         0.16666667 ... 0.         0.25       0.        ]]


In [22]:
train_sales.shape

(1913, 30490)

In [23]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(train_sales, test_size=0.2, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 1530
No. of testing examples: 383


In [24]:
training_data

array([[0.4       , 0.        , 0.        , ..., 0.25      , 0.        ,
        0.08333333],
       [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
        0.        ],
       [0.2       , 0.        , 0.        , ..., 0.1       , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.1       , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
        0.        ],
       [0.        , 0.2       , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [25]:
training_data.shape

(1530, 30490)

In [26]:
#Take the 14 days, predict the 15th
timesteps = 14

X_training_data = []
y_training_data = []

for i in range(timesteps, 1530):
    X_training_data.append(training_data[i-timesteps:i])
    y_training_data.append(training_data[i][0:30490])

In [27]:
#for i in range(timesteps, 1913):
 #   v = train_sales[i-timesteps:i]
  #  u = train_sales[i][0:30490]
   # print('i is: ', (i))
    #print('timestep is: ', (timesteps))
    #break

In [28]:
(training_data[i-timesteps:i])

array([[0.2       , 0.2       , 0.        , ..., 0.3       , 0.        ,
        0.        ],
       [0.2       , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.2       , 0.        , ..., 0.        , 0.08333333,
        0.25      ],
       ...,
       [0.        , 0.2       , 0.        , ..., 0.05      , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.1       , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
        0.        ]])

In [30]:
(training_data[20-timesteps:20])

array([[0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.1 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ],
       ...,
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]])

In [31]:
len((training_data[20-timesteps:20]))

14

In [32]:
 training_data[14][0:30490]

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
X_training_data

[array([[0.4       , 0.        , 0.        , ..., 0.25      , 0.        ,
         0.08333333],
        [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
         0.        ],
        [0.2       , 0.        , 0.        , ..., 0.1       , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.2       , 0.        , ..., 0.25      , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ],
        [0.2 , 0.  , 0.  , ..., 0.1 , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.1 , 0.  , 0.  ],
        ...,
        [0.  , 0.2 , 0.  , ..., 0.25, 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ]]),
 array([[0.2       , 0.        , 0.        , ..., 0.1       , 0.        ,
         0.   

In [34]:
y_training_data

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.        , 0.        , 0.        , ..., 0.1       , 0.08333333,
        0.        ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.        , 0.2       , 0.        , ..., 0.15      , 0.        ,
        0.16666667]),
 array([0.        , 0.        , 0.16666667, ..., 0.        , 0.08333333,
        0.08333333]),
 array([0.4       , 0.        , 0.        , ..., 0.        , 0.16666667,
        0.41666667]),
 array([0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.  , 0.  , 0.  , ..., 0.05, 0.  , 0.  ]),
 array([0. , 0. , 0. , ..., 0.1, 0. , 0. ]),
 array([0.2 , 0.  , 0.  , ..., 0.05, 0.  , 0.  ]),
 array([0. , 0.2, 0. , ..., 0. , 0. , 0. ]),
 array([0.  , 0.2 , 0.  , ..., 0.05, 0.  , 0.  ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.4       , 0.2       , 0.1666666

In [35]:
X_training_data = np.array(X_training_data, dtype='float16')
y_training_data = np.array(y_training_data, dtype='float16')

In [36]:
X_training_data

array([[[0.4   , 0.    , 0.    , ..., 0.25  , 0.    , 0.0833],
        [0.    , 0.    , 0.    , ..., 0.05  , 0.    , 0.    ],
        [0.2   , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
        ...,
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.2   , 0.    , ..., 0.25  , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.    , 0.    , 0.    , ..., 0.05  , 0.    , 0.    ],
        [0.2   , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
        ...,
        [0.    , 0.2   , 0.    , ..., 0.25  , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ]],

       [[0.2   , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        ...,
        [0. 

In [37]:
y_training_data

array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.1   , 0.0833, 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.    , 0.    , 0.    , ..., 0.1   , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.05  , 0.    , 0.    ],
       [0.    , 0.2   , 0.    , ..., 0.    , 0.    , 0.    ]],
      dtype=float16)

In [38]:
print(f'x_train.shape: {X_training_data.shape}')
print(f'y_train.shape: {y_training_data.shape}')

x_train.shape: (1516, 14, 30490)
y_train.shape: (1516, 30490)


# RNN

In [39]:
# model
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.SimpleRNN(units = 64, input_shape = (np.array(X_training_data).shape[1], np.array(X_training_data).shape[2])))
#model.add(tf.keras.layers.Dense(1))

#model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

#model.summary()

In [40]:
# model
model = tf.keras.Sequential()
model.add(tf.keras.layers.SimpleRNN(units = 64))
model.add(tf.keras.layers.Dense(1))

model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

#model.summary()

In [41]:
model.fit(X_training_data, y_training_data, epochs = 10, batch_size = 10)

Epoch 1/10
152/152 [==============================] - 15s 96ms/step - loss: 0.1372
Epoch 2/10
152/152 [==============================] - 15s 99ms/step - loss: 0.0184
Epoch 3/10
152/152 [==============================] - 16s 108ms/step - loss: 0.0170
Epoch 4/10
152/152 [==============================] - 16s 102ms/step - loss: 0.0169
Epoch 5/10
152/152 [==============================] - 15s 102ms/step - loss: 0.0169
Epoch 6/10
152/152 [==============================] - 16s 102ms/step - loss: 0.0169
Epoch 7/10
152/152 [==============================] - 16s 106ms/step - loss: 0.0168
Epoch 8/10
152/152 [==============================] - 16s 106ms/step - loss: 0.0168
Epoch 9/10
152/152 [==============================] - 16s 105ms/step - loss: 0.0168
Epoch 10/10
152/152 [==============================] - 16s 103ms/step - loss: 0.0168


In [42]:
# test dataset

#inputs = train_sales[-timesteps:]
#inputs = sc.transform(inputs)

In [43]:
testing_data

array([[0.        , 0.        , 0.16666667, ..., 0.        , 0.        ,
        0.41666667],
       [0.        , 0.        , 0.16666667, ..., 0.05      , 0.08333333,
        0.08333333],
       [0.        , 0.        , 0.        , ..., 0.1       , 0.        ,
        0.        ],
       ...,
       [0.        , 0.2       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.15      , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.05      , 0.25      ,
        0.41666667]])

In [44]:
testing_data.shape

(383, 30490)

In [45]:
#Take the 14 days, predict the 15th
timesteps = 14

X_testing_data = []
y_testing_data = []

for i in range(timesteps, 383):
    X_testing_data.append(testing_data[i-timesteps:i])
    y_testing_data.append(testing_data[i][0:30490])

In [46]:
X_testing_data

[array([[0.        , 0.        , 0.16666667, ..., 0.        , 0.        ,
         0.41666667],
        [0.        , 0.        , 0.16666667, ..., 0.05      , 0.08333333,
         0.08333333],
        [0.        , 0.        , 0.        , ..., 0.1       , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
         0.        ],
        [0.2       , 0.4       , 0.        , ..., 0.05      , 0.41666667,
         0.33333333]]),
 array([[0.        , 0.        , 0.16666667, ..., 0.05      , 0.08333333,
         0.08333333],
        [0.        , 0.        , 0.        , ..., 0.1       , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.05      , 0.        ,
         0.        ],
        [0.2

In [47]:
#X_test = []
#X_test.append(inputs[0:timesteps])
#X_test = np.array(X_test)

In [48]:
#inputs[0:timesteps]

In [49]:
y_testing_data

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.2       , 1.        , 0.83333333, ..., 0.05      , 0.        ,
        0.25      ]),
 array([0.        , 0.2       , 0.        , ..., 0.        , 0.33333333,
        0.        ]),
 array([0.        , 0.2       , 0.        , ..., 0.1       , 0.08333333,
        0.        ]),
 array([0.2       , 0.        , 0.        , ..., 0.        , 0.        ,
        0.33333333]),
 array([0. , 0. , 0. , ..., 0.1, 0. , 0. ]),
 array([0.4       , 0.        , 0.16666667, ..., 0.05      , 0.        ,
        0.08333333]),
 array([0.        , 0.2       , 0.        , ..., 0.        , 0.        ,
        0.16666667]),
 array([0. , 0.2, 0. , ..., 0. , 0. , 0. ]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0.4, 0. , 0. , ..., 0. , 0. , 0. ]),
 array([0.2, 0. , 0. , ..., 0. , 0. , 0. ]),
 array([0.        , 0.        , 0.        , ..., 0.        , 0.25      ,
        0.08333333]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),

In [50]:
X_testing_data = np.array(X_testing_data, dtype='float16')
y_testing_data = np.array(y_testing_data, dtype='float16')

In [55]:
len(X_testing_data)

369

In [57]:
X_testing_data.shape

(369, 14, 30490)

In [56]:
len(y_testing_data)

369

In [58]:
y_testing_data.shape

(369, 30490)

In [60]:
y_pred = model.predict(X_testing_data)

In [61]:
y_pred

array([[0.0658667 ],
       [0.06463652],
       [0.06401735],
       [0.06556188],
       [0.06594323],
       [0.06232627],
       [0.06610566],
       [0.06514844],
       [0.06596937],
       [0.06351205],
       [0.06203483],
       [0.06247728],
       [0.06552877],
       [0.06443366],
       [0.06544598],
       [0.06281435],
       [0.06407153],
       [0.06258612],
       [0.06375029],
       [0.06307407],
       [0.062081  ],
       [0.06561463],
       [0.06351625],
       [0.06236898],
       [0.06545111],
       [0.06594824],
       [0.06386071],
       [0.06594237],
       [0.06516927],
       [0.06528541],
       [0.06262352],
       [0.06502067],
       [0.06475087],
       [0.0654135 ],
       [0.0643227 ],
       [0.06446107],
       [0.06317323],
       [0.06504097],
       [0.06470709],
       [0.06442665],
       [0.06313699],
       [0.06599086],
       [0.06312918],
       [0.06541001],
       [0.06462278],
       [0.06409162],
       [0.0661289 ],
       [0.065

In [62]:
len(y_pred)

369

In [63]:
y_pred.shape

(369, 1)

In [64]:
np.mean(keras.losses.mean_squared_error(y_testing_data, y_pred))

0.016910113

# LSTM

In [74]:
# model
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.LSTM(units = 64, input_shape = (np.array(X_training_data).shape[1], np.array(X_train).shape[2])))
#model.add(tf.keras.layers.Dense(1))

#model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

#model.summary()

In [71]:
# model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units = 64))
model.add(tf.keras.layers.Dense(1))

model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

In [73]:
model.fit(X_training_data, y_training_data, epochs = 10, batch_size = 10)

Epoch 1/10
152/152 [==============================] - 49s 312ms/step - loss: 0.1828
Epoch 2/10
152/152 [==============================] - 44s 289ms/step - loss: 0.0201
Epoch 3/10
152/152 [==============================] - 45s 296ms/step - loss: 0.0190
Epoch 4/10
152/152 [==============================] - 46s 303ms/step - loss: 0.0180
Epoch 5/10
152/152 [==============================] - 45s 295ms/step - loss: 0.0173
Epoch 6/10
152/152 [==============================] - 43s 286ms/step - loss: 0.0170
Epoch 7/10
152/152 [==============================] - 43s 285ms/step - loss: 0.0169
Epoch 8/10
152/152 [==============================] - 44s 292ms/step - loss: 0.0168
Epoch 9/10
152/152 [==============================] - 45s 298ms/step - loss: 0.0168
Epoch 10/10
152/152 [==============================] - 44s 288ms/step - loss: 0.0168


In [76]:
Y_pred = model.predict(X_testing_data)

In [77]:
Y_pred

array([[0.0676696 ],
       [0.06767997],
       [0.0676703 ],
       [0.06767456],
       [0.06768466],
       [0.06770004],
       [0.06774495],
       [0.06766953],
       [0.0676697 ],
       [0.06767163],
       [0.06766974],
       [0.06767145],
       [0.06768941],
       [0.06767725],
       [0.06768528],
       [0.06793661],
       [0.06786726],
       [0.06767798],
       [0.06767674],
       [0.06769533],
       [0.06769281],
       [0.06769003],
       [0.06767595],
       [0.06770574],
       [0.06768036],
       [0.0676762 ],
       [0.06767829],
       [0.06767313],
       [0.06767021],
       [0.06766981],
       [0.06767227],
       [0.06767024],
       [0.06766994],
       [0.0676734 ],
       [0.06769143],
       [0.06768069],
       [0.06837688],
       [0.06767941],
       [0.0676695 ],
       [0.0676702 ],
       [0.06769843],
       [0.06767474],
       [0.06767933],
       [0.06767006],
       [0.06766991],
       [0.06783607],
       [0.06767787],
       [0.067

In [80]:
np.mean(keras.losses.mean_squared_error(y_testing_data, Y_pred))

0.016908951

# Bidirectional LSTM

In [ ]:
#model = tf.keras.Sequential()
#model.add(tf.keras.layers.Bidirectional(LSTM(units = 64, input_shape = (np.array(X_train).shape[1], np.array(X_train).shape[2]))))
#model.add(tf.keras.layers.Dense(1))

#model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

#model.summary()

In [81]:
# model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Bidirectional(LSTM(units = 64)))
model.add(tf.keras.layers.Dense(1))

model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(0.001))

In [ ]:
model.fit(X_training_data, y_training_data, epochs = 10, batch_size = 10)

Epoch 1/10
152/152 [==============================] - 91s 584ms/step - loss: 0.2864
Epoch 2/10
152/152 [==============================] - 88s 579ms/step - loss: 0.0202
Epoch 3/10
120/152 [======================>.......] - ETA: 18s - loss: 0.0195